In [59]:
import numpy as np
import inspect
import functools

import tensorflow as tf
import gym

import reinforce.config as config
from reinforce.utils import trajectory_dx, partition_rewards, discount_rewards, discount_check
from reinforce.agent import PGAgent, RandomAgent
from reinforce.layers import MLP
from reinforce.train import run_trajectory, rollout

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
#Gym setup
env = gym.make("Pong-v0")

#TF Setup
g = tf.Graph()
sess = tf.Session(graph=g)

#RL Agent Setup
action_params = {
                "input_dim": config.state_dim,
                "hidden_dim": config.hidden_dim,
                "output_dim": config.output_dim
                }
agent = PGAgent(g, sess, config.state_dim, MLP, action_params)
test_agent = RandomAgent()


[2016-09-03 19:50:41,288] Making new env: Pong-v0


In [77]:
_ = agent.rollout(3,env)

In [915]:
experience_buffer = {}

def env_cor(env):
    init_state = env.reset()
    action = yield init_state
    
    try:
        while True:
            state, reward, done, info = env.step(action)
            print "reward {}, done {}".format(reward, done)
            action = yield (state, reward)
            
            if done:
                raise StopIteration
    except TypeError, IndexError:
        print "Action {} not allowed.  Action space is {}.".format(action, env.action_space)
    except GeneratorExit:
        print "Gen exited"
        
def agent_cor(agent):
   
    state = yield
    
    try:
        while True:
            aprob, action = agent.act(state)
            next_state = yield action            
            state = next_state
    except GeneratorExit, StopIteration:
        print "Simulation Finished"
    else:
        raise
        
def simulate(agent, env):
    states, actions, rewards = [], [], []
    
    acor = agent_cor(agent)
    next(acor) #prime 
     
    state = init_state = env.reset()
    action = init_action = acor.send(init_state)
    states.append(state)
    actions.append(action)
    done = False

    while not done:       
        next_state, reward, done, info = env.step(action)          
        rewards.append(reward)
        
        state = next_state
        action = acor.send(state)    
        states.append(state)
        actions.append(action)
        
    acor.close()
    
    return map(np.array, [states, actions, rewards])

In [924]:
states, actions, rewards = simulate(agent, env)
print trajectory_dx(states, actions, rewards)

Simulation Finished
Total actions: 1034
Total reward: -21.0
# of Up moves: 840
# of Down moves: 194
# of positive rewards 0
# of negative rewards 21
None


In [913]:
print len(states), len(actions), len(rewards)

1096 1096 1096


In [70]:
next(acor)


In [882]:
a = acor.send(state)

AttributeError: 'NoneType' object has no attribute 'send'

reward 0.0, done False


In [878]:
while True:
    s, r = ecor.send(a)
    a = acor.send((s,r))

reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward -1.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0, done False
reward 0.0

StopIteration: 

In [75]:
a

3

In [64]:
a

3

In [65]:
o, r, d, i = ecor.send(a)

ValueError: too many values to unpack

In [16]:
def my_coroutine():
    while True:
        received = yield
        print('Received:', received)

In [44]:
dynamics = env_cor(env)

In [48]:
dynamics.close()

Gen exited


In [33]:
dynamics.send(24)

IndexError: index 24 is out of bounds for axis 0 with size 6

In [37]:
next(cor)

In [39]:
cor.send(2)

('Received:', 2)
